### RatU Day3 Two novel recording info

In [2]:
import numpy as np
import subjects
import matplotlib.pyplot as plt
from neuropy import analyses, plotting

sess = subjects.tn.ratUday3[0]
print(sess.recinfo)

filename: /data/Clustering/sessions/RatU/RatUDay3TwoNovel/RatU_Day3TwoNovel_2021-07-27_08-16-07.xml 
# channels: 192
sampling rate: 30000
lfp Srate (downsampled): 1250



### Set probe configuration
- RatU_Day2NSD has two probes: both 128chan-8shanks diagnostic biochips.
- 64 channels in one of the probes (implanted in left hemisphere) had no signal from 4 shanks (probably one of the intan chips was faulty). So only 192 channels were recorded, the channels in .dat file are already order according to depth.


In [ ]:
from neuropy.core import Shank, Probe, ProbeGroup
from neuropy.plotting import plot_probe

shanks = []
channel_groups = sess.recinfo.channel_groups
badchans = sess.recinfo.skipped_channels

# --- diagnostic-biochip 8 shanks -----------
for i in range(8):
    chans = channel_groups[i]
    shank = Shank.auto_generate(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=30,
        y_shift_per_column=[0, -15],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)
    shanks.append(shank)

probe1 = Probe(shanks)

# --- dignostic biochip 4 shanks (4 shanks were bad) -----------
shanks = []
for i in range(8, 12):
    shank = shank.auto_generate(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=30,
        y_shift_per_column=[0, -15],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)
    shanks.append(shank)


probe2 = Probe(shanks)
probe2.move((probe1.x_max + 500, 0))

prbgrp = ProbeGroup()
prbgrp.add_probe(probe1)
prbgrp.add_probe(probe2)

prbgrp.save(sess.filePrefix.with_suffix(".probegroup.npy"))
plot_probe(prbgrp)

### Writing probegroup to json format for spyking-circus 

In [ ]:
from neuropy.utils import probe_util

file = sess.filePrefix.with_suffix(".prb")
probe_util.write_spyking_circus(file, sess.probegroup, combine_shanks=True)

### Experimental paradigm

In [ ]:
import pandas as pd
from neuropy.core import Epoch

datetime_data = pd.read_csv(sess.filePrefix.with_suffix(".datetime.csv"))
durations = datetime_data.nFrames / sess.recinfo.dat_sampling_rate
epochs = pd.DataFrame(
    {
        "start": [0, 9580, 11815, 19405, 23144],
        "stop": [9579, 11814, 19404, 23143, 41758],
        "label": ["pre", "maze1", "post1", "maze2", "post2"],
    }
)

Epoch(epochs=epochs).save(sess.filePrefix.with_suffix(".paradigm.npy"))

### Artifacts epochs

In [ ]:
signal = sess.eegfile.get_signal([20, 170])
artifact_epochs = analyses.detect_artifact_epochs(signal, thresh=8, edge_cutoff=2)
artifact_epochs.save(sess.filePrefix.with_suffix(".artifact.npy"))

In [ ]:
signal = sess.eegfile.get_signal([20])
plotting.plot_artifact_epochs(artifact_epochs, signal)

In [ ]:
from neuropy.io import SpykingCircusIO

file = sess.filePrefix.with_suffix(".dead")
SpykingCircusIO.write_epochs(file, sess.artifact)

### Importing spiketrains from Phy

#### Single folder

In [ ]:
from neuropy.io import PhyIO
from neuropy.core import Neurons
from pathlib import Path
import numpy as np

cluster_path = Path(
    "/home/bapung/Documents/ClusteringHub/spykcirc/RatU/RatUDay3TwoNovel/RatU_Day3TwoNovel_2021-07-27_08-16-07-1.GUI"
)
chan_grps = sess.recinfo.channel_groups
phy_data = PhyIO(cluster_path)
spiketrains = phy_data.spiketrains
peak_chans = phy_data.peak_channels
waveforms = phy_data.waveforms
shank_id = sess.probegroup.get_shank_id_for_channels(peak_chans)

neuron_type_id = phy_data.cluster_info.q.values
neuron_type = np.ones(len(neuron_type_id), dtype="U5")
neuron_type[neuron_type_id < 4] = "pyr"
neuron_type[neuron_type_id == 6] = "mua"
neuron_type[neuron_type_id == 8] = "inter"


neurons = Neurons(
    np.array(spiketrains, dtype=object),
    t_stop=sess.eegfile.duration,
    sampling_rate=phy_data.sampling_rate,
    peak_channels=peak_chans,
    waveforms=np.array(waveforms, dtype="object"),
    neuron_type=neuron_type,
    shank_ids=np.array(shank_id).astype(int),
    metadata={"cluster_path": str(cluster_path)},
)

neurons.save(sess.filePrefix.with_suffix(".neurons"))

### Create MUA

In [3]:
mua = sess.neurons.get_mua()
mua.save(sess.filePrefix.with_suffix(".mua"))

/data/Clustering/sessions/RatU/RatUDay3TwoNovel/RatU_Day3TwoNovel_2021-07-27_08-16-07.mua saved


### Position data

In [ ]:
from neuropy.io import OptitrackIO
from neuropy.core import Position
from pathlib import Path

opti_folder = sess.filePrefix.parent / "position"
opti_data = OptitrackIO(dirname=opti_folder, scale_factor=0.25)

In [ ]:
import pandas as pd
from datetime import datetime

# ------- maze align corection ---------
t_error = [0] * 9

# ---- startimes of concatenated .dat files
tracking_sRate = opti_data.sampling_rate
rec_datetime = pd.read_csv(sess.filePrefix.with_suffix(".datetime.csv"))
data_time = []
for i, file_time in enumerate(rec_datetime["StartTime"]):
    # sync_time = rec_datetime["sync_nframes"][i] / rec_datetime["sync_rate"][i]
    tbegin = datetime.strptime(file_time, "%Y-%m-%d_%H-%M-%S") + pd.Timedelta(
        t_error[i], unit="sec"
    )
    nframes = rec_datetime["nFrames"][i]
    duration = pd.Timedelta(nframes / sess.recinfo.dat_sampling_rate, unit="sec")
    tend = tbegin + duration
    trange = pd.date_range(
        start=tbegin,
        end=tend,
        periods=int(duration.total_seconds() * tracking_sRate),
        inclusive="left",
    )
    data_time.extend(trange)
data_time = pd.to_datetime(data_time)


x, y, z = opti_data.get_position_at_datetimes(data_time)
traces = np.vstack((z, x, y))

position = Position(traces=traces, t_start=0, sampling_rate=opti_data.sampling_rate)
position.save(sess.filePrefix.with_suffix(".position.npy"))


_, ax = plt.subplots()
ax.plot(position.time, position.x)

In [ ]:
ax.plot(position.x, position.y)

### Linearize position

In [ ]:
from neuropy.utils import position_util

for e in ["maze1", "maze2"]:
    maze = sess.paradigm[e].flatten()
    maze_pos = sess.position.time_slice(maze[0], maze[1])
    linear_pos = position_util.linearize_position(maze_pos)
    # e = e.replace('-','')
    linear_pos.save(sess.filePrefix.with_suffix(f".{e}.linear"))

In [ ]:
_, ax = plt.subplots()
ax.plot(sess.maze1.time, sess.maze1.x)
ax.plot(sess.maze2.time, sess.maze2.x)